In [1]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [5]:
project_folder = r"C:\Users\ozgur\Desktop\Final Project\Bitirme Projesi\Fotoğraflar"
folder = os.path.join(project_folder, '5_height_and_congruence_of_the_medial_longitudinal_arch')

IMG_HEIGHT = 64
IMG_WIDTH = 64
dataset = []

In [6]:
def get_images_labels(file_folder):
  for root, directory, filenames in os.walk(file_folder):
    for filename in filenames:
      if filename.endswith("jpg"):
        read_images = cv2.imread(os.path.join(root, filename), 1)  # 0: cv2.IMREAD_GRAYSCALE; 1: cv2.IMREAD_COLOR; -1: cv2.IMREAD_UNCHANGED                                          
        read_images = cv2.resize(read_images, dsize=(IMG_WIDTH, IMG_HEIGHT))  # resizing; interpolation=cv2.INTER_AREA
        read_images = cv2.cvtColor(read_images, cv2.COLOR_BGR2GRAY)  #grayscaling
        label_name = os.path.basename(root)
        grayscale_images = read_images / 255  # normalizing
        dataset.append([grayscale_images, label_name])
  return dataset

In [7]:
def data_preprocessing():
  encoder = LabelEncoder()
  obj_data = get_images_labels(folder)

  image_data = np.array([obj_data[image_loc][0] for image_loc in range(len(obj_data))])
  image_data = image_data.reshape(image_data.shape[0], image_data.shape[1], image_data.shape[2], 1)

  label_data = np.array([obj_data[image_loc][1] for image_loc in range(len(obj_data))])
  label_data = encoder.fit_transform(label_data)

  X_train, X_test, y_train, y_test = train_test_split(image_data, label_data, train_size=0.87, random_state=42)

  X_train = X_train[:20]
  y_train = y_train[:20]

  return X_train, X_test, y_train, y_test

In [ ]:
x_train, x_test, x_val, y_train, y_test, y_val = data_preprocessing()

In [ ]:
print(x_train.shape, x_val.shape)
print(y_train.shape, y_val.shape)

# Print the lengths of the each dataset
print(f"\nX_train: {len(x_train)}")
print(f"y_train: {len(y_train)}")
print(f"X_val: {len(x_val)}")
print(f"y_val: {len(y_val)}")
print(f"x_test: {len(x_test)}")
print(f"y_test: {len(y_test)}")


In [ ]:
def create_model(kernel_size: tuple, input_shape: tuple):
  model = Sequential()
  model.add(Conv2D(IMG_WIDTH,
                  kernel_size=kernel_size,
                  strides=(1,1),
                  padding="same",
                  activation="relu",
                  input_shape=input_shape)           
  )
  model.add(MaxPooling2D((2,2)))
  model.add(Conv2D(IMG_WIDTH/2,
                  kernel_size=kernel_size,
                  strides=(1,1),
                  padding="same",
                  activation="relu",
                  input_shape=input_shape)           
  )

  model.add(Flatten())
  model.add(Dense(64, activation='relu')),
  model.add(Dropout(0.4))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(10, activation='softmax'))

  model.compile(optimizer=optimizers.Adam(lr=0.0001), loss="sparse_categorical_crossentropy", metrics=['accuracy'])
  # tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  return model


In [ ]:
my_model = create_model((3, 3), (IMG_WIDTH, IMG_HEIGHT, 1))
results = my_model.fit(x_train, y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data = (x_val, y_val))


In [ ]:
plt.plot(results.history["loss"], label="Loss")
plt.plot(results.history["val_loss"], label="val_loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
plt.plot(results.history["accuracy"], label="Accuracy")
plt.plot(results.history["val_accuracy"], label="val_accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
# Evaluate the performance
my_model.evaluate(x_test, y_test)

In [ ]:
prediction_result = my_model.predict(x_test[545].reshape(1,IMG_WIDTH,IMG_HEIGHT,1))
plt.imshow(prediction_result)
print(prediction_result)

In [ ]:
predicted_folder = prediction_result.argmax()
predicted_probability = prediction_result.max()


In [ ]:
print(f"This image belongs to folder {predicted_folder} with {predicted_probability}% probability")